In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
raw_data = pd.read_csv('Movie.csv')
raw_data.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
1,9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
2,12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
3,24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"September 21, 2021",1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...
4,27,s28,Movie,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,"September 20, 2021",2010,PG-13,103 min,Comedies,Mourning the loss of their beloved junior high...


In [13]:
print(f'Dữ liệu Netflix có {raw_data.shape[1]} cột và {raw_data.shape[0]} dòng.')

Dữ liệu Netflix có 15 cột và 5277 dòng.


In [3]:
raw_data['date_added']=pd.to_datetime(raw_data['date_added'])
raw_data['year_added']=raw_data['date_added'].apply(lambda x: int(x.year))
raw_data['month_added']=raw_data['date_added'].apply(lambda x: int(x.month))

In [4]:
cast = pd.DataFrame(raw_data['cast'].apply(lambda x: str(x).split(', ')).tolist(),index=raw_data['show_id'])
cast=cast.stack().reset_index().drop('level_1',axis=1)
cast.rename(columns={0:'actor'},inplace=True)

In [5]:
director = pd.DataFrame(raw_data['director'].apply(lambda x: str(x).split(', ')).tolist(),index=raw_data['show_id'])
director=director.stack().reset_index().drop('level_1',axis=1)
director.rename(columns={0:'director'},inplace=True)

In [6]:
country = pd.DataFrame(raw_data['country'].apply(lambda x: str(x).split(', ')).tolist(),index=raw_data['show_id'])
country=country.stack().reset_index().drop('level_1',axis=1)
country.rename(columns={0:'country'},inplace=True)

In [7]:
genre = pd.DataFrame(raw_data['listed_in'].apply(lambda x: str(x).split(', ')).tolist(),index=raw_data['show_id'])
genre=genre.stack().reset_index().drop('level_1',axis=1)
genre.rename(columns={0:'genre'},inplace=True)

In [8]:
temp = cast.merge(director, on=['show_id'], how='inner')
temp1 = temp.merge(country, on=['show_id'], how='inner')
temp2 = temp1.merge(genre, on=['show_id'], how='inner')

In [9]:
data = temp2.merge(raw_data[['show_id','type','title','date_added','release_year','rating','duration',
                             'description','year_added','month_added']],
                             on='show_id',how='inner').iloc[:,[0,5,6,1,2,3,4,7,8,9,10,11,12,13]]

In [10]:
movies = data[data['type']=='Movie']
tv_shows = data[data['type']=='TV Show']
tv=tv_shows['duration'].apply(lambda x:int(str(x).split(' ')[0]))
tv_shows=tv_shows.drop('duration',axis=1)
tv_shows['seasons']=tv
tv_shows.head()

,show_id,type,title,actor,director,country,genre,date_added,release_year,rating,description,year_added,month_added,seasons


In [11]:
movie=movies['duration'].apply(lambda x:int(str(x).split(' ')[0]))
movies=movies.drop('duration',axis=1)
movies['duration']=movie
movies.head()

,show_id,type,title,actor,director,country,genre,date_added,release_year,rating,description,year_added,month_added,duration
0,s8,Movie,Sankofa,Kofi Ghanaba,Haile Gerima,United States,Dramas,2021-09-24,1993,TV-MA,"On a photo shoot in Ghana, an American model s...",2021,9,125
1,s8,Movie,Sankofa,Kofi Ghanaba,Haile Gerima,United States,Independent Movies,2021-09-24,1993,TV-MA,"On a photo shoot in Ghana, an American model s...",2021,9,125
2,s8,Movie,Sankofa,Kofi Ghanaba,Haile Gerima,United States,International Movies,2021-09-24,1993,TV-MA,"On a photo shoot in Ghana, an American model s...",2021,9,125
3,s8,Movie,Sankofa,Kofi Ghanaba,Haile Gerima,Ghana,Dramas,2021-09-24,1993,TV-MA,"On a photo shoot in Ghana, an American model s...",2021,9,125
4,s8,Movie,Sankofa,Kofi Ghanaba,Haile Gerima,Ghana,Independent Movies,2021-09-24,1993,TV-MA,"On a photo shoot in Ghana, an American model s...",2021,9,125


In [12]:
t = movies[(movies['type']!='Unknown')&(movies['director']!='Unknown')].groupby(['type','director']).agg(
    {'show_id':'nunique'}).reset_index().sort_values(
    'show_id',ascending=False).head(80)
t1=movies.merge(t,on=['type','director'],how='inner').groupby(['type','director']).agg(
    {'show_id_x':'nunique'}).reset_index().sort_values('show_id_x',ascending=False).head(15).rename(
    {'show_id_x':'Total Movies'},axis=1)
t1

,type,director,Total Movies
18,Movie,Jan Suter,21
53,Movie,Raúl Campos,19
20,Movie,Jay Karas,15
38,Movie,Marcus Raboy,14
6,Movie,Cathy Garcia-Molina,13
39,Movie,Martin Scorsese,12
77,Movie,Youssef Chahine,12
19,Movie,Jay Chapman,12
68,Movie,Steven Spielberg,11
12,Movie,Don Michael Paul,10


In [14]:
pd.options.display.max_colwidth = 200
title_and_cast_df = raw_data[['director','listed_in']]
title_and_cast_df = title_and_cast_df.dropna()
cast_names_counter = {}
for i, row in title_and_cast_df.iterrows():
    movie_cast = row['director']
    movie_cast_split = movie_cast.split(',')
    movie_cast_stripped = [name.strip() for name in movie_cast_split]
    for name in movie_cast_stripped:
        cast_names_counter[name] = cast_names_counter.get(name,0) + 1
dict(sorted(cast_names_counter.items(),key=lambda item:item[1], reverse = True))

{'Unknown': 92,
 'Jan Suter': 21,
 'Raúl Campos': 19,
 'Jay Karas': 15,
 'Marcus Raboy': 14,
 'Cathy Garcia-Molina': 13,
 'Youssef Chahine': 12,
 'Martin Scorsese': 12,
 'Jay Chapman': 12,
 'Steven Spielberg': 11,
 'Don Michael Paul': 10,
 'David Dhawan': 9,
 'Yılmaz Erdoğan': 9,
 'Shannon Hartman': 9,
 'Robert Rodriguez': 8,
 'Hakan Algül': 8,
 'Hanung Bramantyo': 8,
 'Troy Miller': 8,
 'Anurag Kashyap': 8,
 'Justin G. Dyck': 8,
 'Johnnie To': 8,
 'Umesh Mehra': 8,
 'Toshiya Shinohara': 7,
 'Lasse Hallström': 7,
 'Dibakar Banerjee': 7,
 'Clint Eastwood': 7,
 'Ram Gopal Varma': 7,
 'Quentin Tarantino': 7,
 'McG': 7,
 'Ron Howard': 7,
 'Leslie Small': 7,
 'Kunle Afolayan': 7,
 'Lance Bangs': 7,
 'Noah Baumbach': 7,
 'Ryan Polito': 7,
 'Robert Luketic': 6,
 'Steve Brill': 6,
 'David Fincher': 6,
 'Kevin Smith': 6,
 'Ozan Açıktan': 6,
 'Ashutosh Gowariker': 6,
 'Mae Czarina Cruz': 6,
 'Fernando Ayllón': 6,
 'Rajkumar Santoshi': 6,
 'Imtiaz Ali': 6,
 'Anees Bazmee': 6,
 'Jesse V. Johnson':